<a href="https://colab.research.google.com/github/vchiang001/Intro2BehvRes/blob/main/Using_DeepLabCut_screen_camera_trap_images_with_physical_interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using DeepLabCut to filter out MegaDetector analysed camera trap images that show physically interacting animals**

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1616492373700-PGOAC72IOB6AUE47VTJX/ke17ZwdGBToddI8pDm48kB8JrdUaZR-OSkKLqWQPp_YUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYwL8IeDg6_3B-BRuF4nNrNcQkVuAT7tdErd0wQFEGFSnBqyW03PFN2MN6T6ry5cmXqqA9xITfsbVGDrg_goIDasRCalqV8R3606BuxERAtDaQ/modelzoo.png?format=1000w)

The understanding of sexual diversity including the sociosexual interactions between organisms requires consideration of the natural milieu of these behaviours , as well as the evolutionary perspectives by comparing across species. Camera trap data is pivotal for this purpose to look at these behaviours in wildlife animals, and to understand their context (e.g. where and when). For more details, you can refer to this presentation during the 2022 DeepLabCut AI Residency Program at EPFL: https://zenodo.org/record/7040375#.YyJ9a3bMK39 

Our notebook runs on camera trap images that have been analysed using MegaDetector. More details on MegaDetector here: https://github.com/microsoft/CameraTraps/blob/main/megadetector.md 

Our notebook helps you isolate camera trap images that exhibit physical interactions. This is helpful because scientists invest a huge amount of time reviewing camera trap images, and a huge amount of that time is spent reviewing images they aren’t interested in (e.g. empty images, noise). For those interested in sexual behaviour, this can filter out a substantial amount of images, to visually inspect those images with physical interaction, whether sexual behaviour could be occurring. 

For this work, we will use DeepLabCut from the model zoo. More details here: http://modelzoo.deeplabcut.org Please cite the relevant papers for the models you use, and consider giving back by helping to label more data. More details here: https://contrib.deeplabcut.org/ 

- **What you need:**camera trap images, and the JSON output file from MegaDetector analysis (ensure your camera trap images retain the same name in the JSON output file)

- **What to do:** (1) in the top right corner, click "CONNECT". Then, just hit run (play icon) on each cell below and follow the instructions!


## **Let's get going: install DeepLabCut into COLAB:**
*Also, be sure you are connected to a GPU: go to menu, click Runtime > Change Runtime Type > select "GPU"*

In [ ]:
#click the play icon (this will take a few minutes to install all the dependences!)
!pip install deeplabcut-live
!pip install deeplabcut

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)**

## Now let's set the backend & import the DeepLabCut package:

In [ ]:
import os

# stifle tensorflow warnings, like we get it already.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import deeplabcut

##Link your Google Drive (containing camera trap images & JSON file):

### First, place your camera trap image folder & JSON file into you google drive! 

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:

from google.colab import drive
drive.mount('/content/drive')

YOU WILL NEED TO EDIT THE FILE PATH & JSON PATH TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/CameraTrapImages


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:
file_path = '/content/drive/My Drive/CameraTrapImages'
json_path = '/content/drive/My Drive/CameraTrapImages/megadetector.json'

## Select your model from the dropdown menu and set the model for analysis

In [ ]:
import ipywidgets as widgets
from IPython.display import display

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description="Choose a DLC ModelZoo model!",
    disabled=False
)
display(model_selection)

In [ ]:
model2use = model_selection.value
from deeplabcut.utils import auxfun_models
auxfun_models.download_model(modelname = model2use, target_dir = file_path)

YOU WILL NEED TO EDIT THE MODEL PATH & POSE CONFIG PATH TO BE SET TO YOUR GOOGLE DRIVE LINK!

This is currently set in your "file_path", so typically, this will be: /content/drive/My Drive/CameraTrapImages

In [ ]:
#Setup where your model is
# PLEASE EDIT THESE:
model_path = '/content/drive/My Drive/CameraTrapImages'
pose_cfg_path = model_path + '/pose_cfg.yaml' #copy the path to the pose_cfg.yaml 

## This function crops the bounding box from the JSON output

In [ ]:
import PIL
from PIL import Image, ImageFile, ImageFont, ImageDraw
import numpy as np
from tensorflow.python.eager.context import global_seed
def crop_detections(detect, img, imageWidth, imageHeight):
    x1, y1,w_box, h_box = detect["bbox"]
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box 
    area = (xmin * imageWidth, 
            ymin * imageHeight, 
            xmax * imageWidth,
            ymax * imageHeight)
    cropping = img.crop(area)
    list_np_crop = np.asarray(cropping)
    return list_np_crop 

## This function runs the DeepLabCut model on individual MegaDetector bounding box

In [ ]:
from dlclive import DLCLive, Processor

def predict_dlc(model_path, list_np_crop):
    #list_np_crop = np.array(list_np_crop)
    dlc_proc = Processor()
    dlc_live = DLCLive(model_path, processor=dlc_proc)
    dlc_live.init_inference(list_np_crop)
    keypts_xyp = dlc_live.get_pose(list_np_crop) # third column is llk!
    return keypts_xyp


## This function converts the DeepLabCut poses normalised to the original image

In [ ]:
import yaml
from ruamel.yaml import YAML
import pandas as pd 
def pose_orig_img(keypts_xyp, detect, pose_cfg_path):
    x1, y1, w_box, h_box = detect["bbox"] 
    keypts_xyp_orig_img = []
    for x, y, p in keypts_xyp:
        c = [x+x1, y+y1] 
        keypts_xyp_orig_img.append(c)
    with open(pose_cfg_path, "r") as stream:
        pose_cfg_dict = yaml.safe_load(stream)
    bodypart_label = pose_cfg_dict['all_joints_names']
    keypts_xyp_orig_img_df = pd.DataFrame (keypts_xyp_orig_img, columns = ['x', 'y'])
    keypts_xyp_orig_img_df['bodypart'] = bodypart_label
    keypts_xyp_orig_img_df = keypts_xyp_orig_img_df.set_index('bodypart')
    return keypts_xyp_orig_img_df

## This function obtains the reference length for each image (as each image differ in size) 

In [ ]:
from scipy.spatial import distance
from statistics import mean

def nose2eye(allposes_names):
  # get reference distance 
  eye_names = ['L_Eye', 'R_Eye', 'r_eye', 'l_eye', 'forehead', 'left_eye', 'right_eye']
  nose_names = ['Nose', 'nose', 'chin'] 
  nose2eye_dist = [] 
  arbr_ref = float(4) #this is currently set as 4 arbitrarily if no nose and eyes are detected 
  nose2eye_dist.append(arbr_ref)    
  for entity in allposes_names: 
      eye_list = [] 
      df_body = globals()[entity]
      for body in df_body.index: 
          if body in eye_names: 
              eye_list.append(list(df_body.loc[body])) 
          if len(eye_list) != 0: 
              nose_list = [] 
              for body in df_body.index:
                  if body in nose_names:
                      nose_list.append(list(df_body.loc[body])) 
      for eye in eye_list: 
          dist = distance.euclidean(eye,nose_list)
          nose2eye_dist.append(dist) 
  ref = min(nose2eye_dist)
  return ref 

## This function checks if there is physical interaction between individual MegaDetector boxes in each image

In [ ]:
import pandas as pd 
def phys_int(allposes_names, ref):
    int_matrx = [] 
    for entity in allposes_names: 
        df_body = globals()[entity]
        xy = df_body.values.tolist() 
        for entity in allposes_names:
            df_body2 = globals()[entity]
            ab = df_body2.values.tolist() 
            for i in xy: 
                for b in ab: 
                    dist = distance.euclidean(i, b)
                    if dist != 0: 
                        int_matrx.append(dist) 
    if any(i < ref for i in int_matrx): 
        social = "physically interacting"
    else:
      social = "not physically interacting"
    return social 

## After loading all the functions, we are ready to run! 

This creates a separate folder, and all camera trap images that exhibit physical interactions will be copied into the folder. \
Currently this is set to only the MegaDetector bounding boxes with confidence above 0.8, but this can be adjusted. 

In [ ]:
# open json file
import json 
with open(json_path, 'r') as f:
    detection_results = json.load(f)

# this is used to temporary name bounding boxes
import random 
import string  

# creates a new folder to copy camera trap images with physical interactions
import glob
import shutil
import os
phys_int_path = os.path.join(file_path, "phys_int_camtrap")
os.mkdir(phys_int_path) 

# this runs through all images in the JSON file
for img_data in detection_results["images"]:
    img_path =  os.path.join(file_path, img_data['file'])
    img = Image.open(img_path)
    imageWidth = img.size[0]
    imageHeight = img.size[1]
    if len(img_data["detections"]) >= 2: 
        allposes_names = [] 
        for detect in img_data["detections"]: 
            if detect['conf'] > 0.8: 
                list_np_crop = crop_detections(detect, img, imageWidth, imageHeight)
                keypts_xyp = predict_dlc(model_path, list_np_crop) 
                keypts_xyp_orig_img_df = pose_orig_img(keypts_xyp, detect, pose_cfg_path)
                letters = string.ascii_letters 
                name =  ''.join(random.choice(letters) for i in range(10)) 
                allposes_names.append(name) 
                locals()[name] = keypts_xyp_orig_img_df
        ref =  nose2eye(allposes_names)
        social = phys_int(allposes_names, ref)
        file = img_data['file'] 
        if social == "physically interacting":
            shutil.copy(file_path+'/'+file, phys_int_path) 

###Happy DeepLabCutting! Welcome to the Zoo :)